In [1]:
import csv
import json
import os

import requests

In [2]:
# Create the chart/map with post request

def create_chart(
    title,
    url: str = 'https://api.datawrapper.de/v3/charts',
    map_type: str = 'd3-maps-choropleth',
    token: str = os.getenv('DATAWRAPPER_ACCESS_TOKEN'),
    ):
    
    headers = {
        'Authorization': 'Bearer ' + token,
        'Content-Type': 'application/json',
        }

    data = {
        "title": title,
        "type": map_type,
    }

    r = requests.post(url, headers=headers, json=data)

    return r

r = create_chart('Organic farms in Europe')

In [3]:
def get_chart_id(r: requests.Response):
    return json.loads(r.text)['publicId']

chart_id = get_chart_id(r)

In [4]:
def get_visualize_url(r):
    return f'https://app.datawrapper.de/map/{chart_id}/visualize'

print(get_visualize_url(r))

https://app.datawrapper.de/map/kBxZS/visualize


In [5]:
def get_csv_data(filename: str):
    with open('organic_share.csv') as f:
        data = f.read()
    return data

data = get_csv_data('organic_share.csv')
data

'geo,organic_share\nAT,25.33\nBE,6.85\nBG,2.34\nCH,11.18\nCY,4.98\nCZ,15.19\nDE,7.75\nDK,11.09\nEE,22.33\nES,9.66\nFI,13.48\nFR,7.72\nGB,2.62\nGR,10.26\nHR,7.19\nHU,5.71\nIE,1.63\nIS,0.37\nIT,15.16\nLT,8.14\nLU,4.42\nLV,14.79\nMK,0.29\nMT,0.47\nNL,3.75\nNO,4.62\nPL,3.49\nRO,2.86\nRS,0.61\nSE,20.43\nSI,10.35\nSK,10.31\nTR,1.46\n'

In [6]:
# Upload data the data with put request
def put_data(
    chart_id: str,
    data: str,
    url = f'https://api.datawrapper.de/v3/charts/{chart_id}/data',
    token: str = os.getenv('DATAWRAPPER_ACCESS_TOKEN'),
    ):
    headers = {
        'Authorization': 'Bearer ' + token,
        'Content-Type': 'text/csv',
        }

    r = requests.put(url, headers=headers, data=data)
    return r

r = put_data(chart_id, data)

In [7]:
# Select correct basemap, country codes and values to plot
def patch_chart_data(
    chart_id: str,
    data: str,
    url = f'https://api.datawrapper.de/v3/charts/{chart_id}',
    token: str = os.getenv('DATAWRAPPER_ACCESS_TOKEN'),
    ):
    headers = {
        'Authorization': 'Bearer ' + token,
        'Content-Type': 'application/json',
        }

    r = requests.patch(url, headers=headers, json=data)
    return r

data = {
    "metadata": {
      "axes": {
          "keys": "geo",
          "values": "organic_share",
          "labels": "organic_share",
      },
      "visualize": {
          "basemap": "europe",
          "map-key-attr": "ISO_A2"
      }
    }
  }

r = patch_chart_data(chart_id, data)

In [8]:
# Before updating the looks check current settings
# Get data about the chart
def get_chart_metadata_asdict(
    chart_id: str,
    url = f'https://api.datawrapper.de/v3/charts/{chart_id}',
    token: str = os.getenv('DATAWRAPPER_ACCESS_TOKEN'), 
    ):
    url = f'https://api.datawrapper.de/v3/charts/{chart_id}'

    headers = {
        'Authorization': 'Bearer ' + token,
        'Content-Type': 'application/json',
    }

    r = requests.get(url, headers=headers)
    return json.loads(r.text)
    

metadata = get_chart_metadata_asdict(chart_id)

In [9]:
# Patch labels: make them visible and use column organic_share with 0.0% format
# Patch colorscheme and tooltips
data = {
    "metadata": {
        "visualize": {
          "colorscale": {
              'colors': [{'color': '#fefaca', 'position': 0},
                 {'color': '#008b15', 'position': 1}],
          },
            "map-label-format": "0.0%",
            "labels": {
                "enabled": True,
                "type": "column"
            },
        "legend": {
            "labelFormat": "0.0%"
        },
        "tooltip": {
            "body": '{{ organic_share }}%',
            'title': '%REGION_NAME%',
            'enabled': True,
        }
        },
        "describe": {
            "source-name": "Eurostat",
            "source-url": "https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Organic_farming_statistics",
            "intro": "Share of organic farms area in total utilised agricultural area in 2019"
        }
    }
    
  }

r = patch_chart_data(chart_id, data)

In [10]:
# Publish the chart
def publish_chart(
    chart_id: str,
    url =  f'https://api.datawrapper.de/v3/charts/{chart_id}/publish',
    ):

    headers = {
        'Authorization': 'Bearer ' + os.getenv('DATAWRAPPER_ACCESS_TOKEN'),
    }

    r = requests.post(url, headers=headers)
    return r

r = publish_chart(chart_id)

In [11]:
def get_chart_url(chart_id):
    return f'https://www.datawrapper.de/_/{chart_id}/'

print(get_chart_url(chart_id))

https://www.datawrapper.de/_/kBxZS/


In [12]:
def chart_to_png(
    chart_id: str,
    params: dict,
    output: str,
    url: str =  f'https://api.datawrapper.de/v3/charts/{chart_id}/export/png',
    ):
    
    headers = {
        'Authorization': 'Bearer ' + os.getenv('DATAWRAPPER_ACCESS_TOKEN'),
        'Accept': '*/*',
    }
    
    r = requests.get(url, headers=headers, params=params)
    
    with open(output, "wb") as f:
        f.write(r.content)
    
    return r

params = {"unit":"px","mode":"rgb","plain":"false","scale":"1","zoom":"1","download":"false","fullVector":"false","ligatures":"true","borderWidth": 20, "transparent":"false","logo":"auto"}

chart_to_png(chart_id, params=params, output='map.png')


<Response [200]>

<iframe title="Organic farms in Europe" aria-label="Map" id="datawrapper-chart-gxcYi" src="https://datawrapper.dwcdn.net/gxcYi/1/" scrolling="no" frameborder="0" style="width: 0; min-width: 100% !important; border: none;" height="undefined"></iframe><script type="text/javascript">!function(){"use strict";window.addEventListener("message",(function(e){if(void 0!==e.data["datawrapper-height"]){var t=document.querySelectorAll("iframe");for(var a in e.data["datawrapper-height"])for(var r=0;r<t.length;r++){if(t[r].contentWindow===e.source)t[r].style.height=e.data["datawrapper-height"][a]+"px"}}}))}();
</script>

<iframe title="Organic farms in Europe" aria-label="Map" id="datawrapper-chart-gxcYi" src="https://datawrapper.dwcdn.net/gxcYi/1/" scrolling="no" frameborder="0" style="width: 0; min-width: 100% !important; border: none;" height="undefined"></iframe><script type="text/javascript">!function(){"use strict";window.addEventListener("message",(function(e){if(void 0!==e.data["datawrapper-height"]){var t=document.querySelectorAll("iframe");for(var a in e.data["datawrapper-height"])for(var r=0;r<t.length;r++){if(t[r].contentWindow===e.source)t[r].style.height=e.data["datawrapper-height"][a]+"px"}}}))}();
</script>